In [ ]:
#Authentication
from azure.quantum import Workspace
workspace = Workspace (
   resource_id = "###Insert your workspace credentials here###",
   location = "###Insert Location###"
)


In [ ]:
#Imports
from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor
from azure.quantum.qiskit import AzureQuantumProvider

In [ ]:
#define the provider
provider = AzureQuantumProvider(
  resource_id="###insert workspace###",
  location="###insert location###"
)

First let's see which backends are available on Azure Quantum

In [ ]:
print("This workspace's targets:")
for backend in provider.backends():
    print("- " + backend.name())

In [ ]:
#Here we can check what backends are available on our workspace
print([backend.name() for backend in provider.backends()])

Create the first bell pair $\phi^+ = \frac{1}{2}(\ket{00}+\ket{11})$

In [ ]:
# Create a Quantum Circuit with 2 qubits initialized |00> and 2 classical bits to store the results
circuit = QuantumCircuit(2, 2)
circuit.name = "My first entangled pair"

# Print out the circuit
circuit.draw()

In [ ]:
#Apply the Hadamard Gate to the first Qubit

circuit.h(0)

# Print out the circuit
circuit.draw()

In [ ]:
#Apply the CNOT gate to the qubit 1 and 2

circuit.cx(0,1)

# Print out the circuit
circuit.draw()

In [ ]:
#Now we want to measure our result

circuit.measure([0,1], [0,1])

#Note the measurement is in the computational basis {|0>, |1>}
# Print out the circuit
circuit.draw()

In [ ]:
circuit = QuantumCircuit(2, 2)
circuit.name = "My first entangled pair"

circuit.h(0)
circuit.cx(0,1)

circuit.measure([0,1], [0,1])

# Print out the circuit
circuit.draw()

In [ ]:
#define the backend to run the circuit
simulator_backend = provider.get_backend("ionq.simulator") #here we set as default the IonQ simulator

In [ ]:
#Want to change backend? Uncomment the next line. Note: It might take some time if you change backend
#simulator_backend = provider.get_backend("quantinuum.sim.h1-1e")

In [ ]:
#create the job to send to Azure
job = simulator_backend.run(circuit, shots=100)
job_id = job.id()
print("Job id", job_id)

In [ ]:
result = job.result()
print(result)

In [ ]:
counts = {format(n, "02b"): 0 for n in range(4)}
counts.update(result.get_counts(circuit))
print(counts)
plot_histogram(counts)

Let's now create the other bell pairs 
$\\\phi^- = \frac{1}{2}(\ket{00}-\ket{11})$
$\\\psi^+ = \frac{1}{2}(\ket{01}+\ket{10})$
$\\\psi^- = \frac{1}{2}(\ket{01}-\ket{10})$



In [ ]:
#phi - 
circuit = QuantumCircuit(2, 2)
circuit.name = "phi_minus"

###Insert the gates here###

circuit.measure([0,1], [0,1])

# Print out the circuit
circuit.draw()

In [ ]:
job = simulator_backend.run(circuit, shots=100)
result = job.result()
counts = {format(n, "02b"): 0 for n in range(4)}
counts.update(result.get_counts(circuit))
print(counts)
plot_histogram(counts)

In [ ]:
#psi + 
circuit = QuantumCircuit(2, 2)
circuit.name = "phi_minus"

###Insert the gates here###

circuit.measure([0,1], [0,1])

# Print out the circuit
circuit.draw()

In [ ]:
job = simulator_backend.run(circuit, shots=100)
result = job.result()
counts = {format(n, "02b"): 0 for n in range(4)}
counts.update(result.get_counts(circuit))
print(counts)
plot_histogram(counts)


In [ ]:
#psi - 
circuit = QuantumCircuit(2, 2)
circuit.name = "phi_minus"

###Insert the gates here###

circuit.measure([0,1], [0,1])

# Print out the circuit
circuit.draw()

In [ ]:
job = simulator_backend.run(circuit, shots=100)
result = job.result()
counts = {format(n, "02b"): 0 for n in range(4)}
counts.update(result.get_counts(circuit))
print(counts)
plot_histogram(counts)

In [ ]:
###Select a qpu backend provider###
qpu_backend= provider.get_backend("ionq.qpu") 

Optional, create a script that takes in input the bell pair we want to build and create the circuit

In [ ]:
#Run on actual QPU
# Submit the circuit to run on Azure Quantum
qpu_job = qpu_backend.run(circuit, shots=1024)
job_id = qpu_job.id()
print("Job id", job_id)

# Monitor job progress and wait until complete:
job_monitor(qpu_job)

# Get the job results (this method also waits for the Job to complete):
result = qpu_job.result()
print(result)
counts = {format(n, "02b"): 0 for n in range(4)}
counts.update(result.get_counts(circuit))
print(counts)
plot_histogram(counts)

Use Microsoft Resource Estimator

In [ ]:
from azure.quantum.qiskit import AzureQuantumProvider
from qiskit import QuantumCircuit, transpile
from qiskit.tools.monitor import job_monitor

In [ ]:
from azure.quantum import Workspace
from azure.quantum.qiskit import AzureQuantumProvider

workspace = Workspace(
            resource_id = "/subscriptions/799529d3-173c-4965-a9c5-b2d64bf4ab95/resourceGroups/NT_Prototyping/providers/Microsoft.Quantum/Workspaces/QuantumDemo",
            location = "westeurope")


provider = AzureQuantumProvider(workspace)

In [ ]:
backend = provider.get_backend('microsoft.estimator')

In [ ]:
def create_algorithm(circ, backend):
    circuit = circ

    # One could further reduce the resource estimates by increasing the optimization_level,
    # however, this will also increase the runtime to construct the algorithm.  Note, that
    # it does not affect the runtime for resource estimation.
    print(f"[INFO] Decompose circuit into intrinsic quantum operations")

    # retrieve basis gates from backend
    basis_gates = backend.configuration().basis_gates
    circuit = transpile(circuit, basis_gates=basis_gates, optimization_level=0)

    # print some statistics
    print(f"[INFO]   qubit count: {circuit.num_qubits}")
    print("[INFO]   gate counts")
    for gate, count in circuit.count_ops().items():
        print(f"[INFO]   - {gate}: {count}")

    return circuit    

In [ ]:
circ = create_algorithm(circuit, backend)


In [ ]:
job = backend.run(circ)
job_monitor(job)
result = job.result()

In [ ]:
result

In [ ]:
result.diagram.space

Time

In [ ]:
result.summary

In [ ]:
#Working on changing the parameters
job = backend.run(circ,
    qubitParams={
        "name": "qubit_maj_ns_e6"
    },
    qecScheme={
        "name": "floquet_code"
    })
job_monitor(job)
result_maj_floquet = job.result()
result_maj_floquet

Using bell pairs to create superdense coding

In [ ]:
def create_bell_pair():
    """
    Returns:
        QuantumCircuit: Circuit that produces a Bell pair
    """
    qc = QuantumCircuit(2)
    qc.h(1)
    qc.cx(1, 0)
    return qc

In [ ]:
def encode_message(qc, qubit, msg):
    """Encodes a two-bit message on qc using the superdense coding protocol
    Args:
        qc (QuantumCircuit): Circuit to encode message on
        qubit (int): Which qubit to add the gate to
        msg (str): Two-bit message to send
    Returns:
        QuantumCircuit: Circuit that, when decoded, will produce msg
    Raises:
        ValueError if msg is wrong length or contains invalid characters
    """
    if len(msg) != 2 or not set(msg).issubset({"0","1"}):
        raise ValueError(f"message '{msg}' is invalid")
    if msg[1] == "1":
        qc.x(qubit)
    if msg[0] == "1":
        qc.z(qubit)
    return qc

In [ ]:
def decode_message(qc):
    qc.cx(1, 0)
    qc.h(1)
    return qc

In [ ]:
# Charlie creates the entangled pair between Alice and Bob
qc = create_bell_pair()

# We'll add a barrier for visual separation
qc.barrier()

# At this point, qubit 0 goes to Alice and qubit 1 goes to Bob

# Next, Alice encodes her message onto qubit 1. In this case,
# we want to send the message '10'. You can try changing this
# value and see how it affects the circuit
message = '10'
qc = encode_message(qc, 1, message)
qc.barrier()
# Alice then sends her qubit to Bob.

# After recieving qubit 0, Bob applies the recovery protocol:
qc = decode_message(qc)

# Finally, Bob measures his qubits to read Alice's message
qc.measure_all()

# Draw our output
qc.name = "SuperdenseCoding"
qc.draw()

In [ ]:
j = simulator_backend.run(qc, shots=200)
j_id = j.id()
print("Job id", j_id)
result = j.result()
counts = {format(n, "02b"): 0 for n in range(4)}
counts.update(result.get_counts(qc))
print(counts)
plot_histogram(counts)


In [ ]:
#Make it a function
def superdensecoding(msg : str):
    qc = create_bell_pair()
    qc.barrier()
    message = msg
    qc = encode_message(qc, 1, message)
    qc.barrier()
    qc = decode_message(qc)
    qc.measure_all()
    qc.draw()    
    j = simulator_backend.run(qc, shots=200)
    result = j.result()
    counts = {format(n, "02b"): 0 for n in range(4)}
    counts.update(result.get_counts(qc))
    return(counts)

In [ ]:
msg = '11'
print(superdensecoding(msg))
plot_histogram(superdensecoding(msg))

In [ ]:
#let's estimate the resources needed to run this application
sdcirc = transpile(qc, basis_gates=backend.configuration().basis_gates, optimization_level=0)

In [ ]:
job = backend.run(sdcirc)
job_monitor(job)
result = job.result()
result